In [2]:
import pandas as pd

In [3]:
data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [4]:
prices

,product_id,start_date,end_date,price
0,1,2019-02-17,2019-02-28,5
1,1,2019-03-01,2019-03-22,20
2,2,2019-02-01,2019-02-20,15
3,2,2019-02-21,2019-03-31,30


In [5]:
units_sold

,product_id,purchase_date,units
0,1,2019-02-25,100
1,1,2019-03-01,15
2,2,2019-02-10,200
3,2,2019-03-22,30


Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places. If a product does not have any sold units, its average selling price is assumed to be 0.

Return the result table in any order.

The result format is in the following example.

| product_id | average_price |
|------------|---------------|
| 1          | 6.96          |
| 2          | 16.96         |

In [6]:
# First join the two tables based on the product id.
prices_sold=prices.merge(right=units_sold, on="product_id", how="left")
# Next filter the sold date on the start and end date.
filtered = prices.merge(right=units_sold, on="product_id", how="left")[(prices_sold["purchase_date"] >= prices_sold["start_date"]) & (prices_sold["purchase_date"] <= prices_sold["end_date"])]

# Create a new column so we can do average later.
filtered["value"] = filtered["price"] * filtered["units"]

# Do sum on units and value then calc the average.
aggregated=filtered[["product_id","units","value"]].groupby(by="product_id").sum().reset_index()
aggregated["average_price"]=aggregated["value"] / aggregated["units"]

# Round and remove unneeded rows.
aggregated=aggregated.round({"average_price":2}).drop(labels=["units","value"],axis=1)

# Get list of unique product_id.
products=prices[["product_id"]].drop_duplicates()

# Left join and replace null with 0.
products.merge(right=aggregated,on="product_id",how="left").fillna(0)

,product_id,average_price
0,1,6.96
1,2,16.96
